In [1]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F

sys.path.append("../src/SpaceMAP/")
from _spacemap import SpaceMAP

In [2]:
with open('../outputs/data-split/latent_idx2carfolder.pickle', 'rb') as handle:
    latent_idx2carfolder = pickle.load(handle)

latent_codes = torch.load('../outputs/latent-codes/latent_codes.pth')

np_latent_codes = latent_codes.cpu().detach().numpy()

In [3]:
def sort_carimg_path(img_paths):
    sorted_img_paths = [None for _ in range(len(latent_idx2carfolder.keys()))]
    for i in range(len(latent_idx2carfolder.keys())):
        carname = latent_idx2carfolder[i].split('/')[-2]
        car_imgpatgh = '../../../data/img/{}.jpg'.format(carname)
        sorted_img_paths[i] = car_imgpatgh

    assert None not in sorted_img_paths, "Some car images are missing"

    return sorted_img_paths

car_imgs = glob('../data/img/**/*.jpg', recursive=True)
car_imgs = sort_carimg_path(car_imgs)

In [4]:
# let's see the decomposed latent space

# 1. PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
reduced_latent_codes = pca.fit_transform(np_latent_codes)
np.save('../outputs/decompose-analysis/latent-codes/reduced_latent_codes-PCA.npy', reduced_latent_codes)

# 2. tSNE
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, perplexity=30.0, n_iter=1000)
reduced_latent_codes = tsne.fit_transform(np_latent_codes)
np.save('../outputs/decompose-analysis/latent-codes/reduced_latent_codes-tSNE.npy', reduced_latent_codes)

# 3. UMAP
from umap import UMAP
umap = UMAP(n_neighbors=15, min_dist=0.1, metric='euclidean')
reduced_latent_codes = umap.fit_transform(np_latent_codes)
np.save('../outputs/decompose-analysis/latent-codes/reduced_latent_codes-UMAP.npy', reduced_latent_codes)


# 4. SpaceMAP
spacemap = SpaceMAP(verbose=False)
reduced_latent_codes = spacemap.fit_transform(np_latent_codes)
np.save('../outputs/decompose-analysis/latent-codes/reduced_latent_codes-spaceMAP.npy', reduced_latent_codes)


[SpaceMAP] start. time: Sun Nov 26 16:10:56 2023
[SpaceMAP] use FAISS library to calculate knn graph
d-local = auto
d-local =  [13.399251  13.40229   16.797054   9.755467   4.8314643  9.716446
 22.113104  14.789581  13.969586  19.047     14.825744  13.041618
 13.141944  25.47139   15.355839  12.985004   6.39115    6.2092333
  4.9484997  9.670938  28.953169  13.247958  13.33704    6.7139587
 18.159365   7.1856327  9.131258  14.50213    8.315401  17.195341
 13.713201   3.478764  14.942944  17.030254  11.485844  10.925143
 16.439102   5.206076   9.642709  11.619477  15.825526  16.301174
  7.860712  18.340961  12.813912  10.845339  11.98323   20.922518
  3.4439185  6.9409757 15.136967  13.413442  11.570713  15.448386
 17.6456    13.318101  16.513935  18.669895  14.449346  11.697479
 14.624166  14.567035  15.454961  25.014654  16.653358  24.531956
  8.1941395 11.093661  17.283432   4.760117  14.478277  19.446587
  8.987748   4.7497616 23.313576   6.0453405 17.425173  25.965591
 13.181106  1

## Graph Plot

In [5]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper

methods = [
    'spaceMAP',
    'UMAP',
    'PCA',
    'tSNE',
]

for method in methods:
    reduced_latent_codes = np.load(f'../outputs/decompose-analysis/latent-codes/reduced_latent_codes-{method}.npy')

    digits_df = pd.DataFrame(reduced_latent_codes, columns=['x', 'y'])
    digits_df['image'] = car_imgs
    digits_df['digit'] = digits_df['image'].apply(lambda x: x.split('/')[-1].split('.')[0])

    datasource = ColumnDataSource(digits_df)

    #color_mapping = CategoricalColorMapper(factors=[str(9 - x) for x in digits.target_names],
    #                                       palette=Spectral10)

    plot_figure = figure(
        title='{} projection of the Digits dataset'.format(method),
        outer_width=600,
        outer_height=600,
        tools=('pan, wheel_zoom, reset')
    )

    plot_figure.add_tools(HoverTool(tooltips="""
    <div>
        <div>
            <img src='@image' style='float: left; margin: 5px 5px 5px 5px; width:300px;'/>
        </div>
        <div>
            <span style='font-size: 16px; color: #224499'>Digit:</span>
            <span style='font-size: 18px'>@digit</span>
        </div>
    </div>
    """))

    plot_figure.circle(
        'x',
        'y',
        source=datasource,
        # color=dict(field='digit', transform=color_mapping),
        line_alpha=0.6,
        fill_alpha=0.6,
        size=4
    )
    show(plot_figure)

    from bokeh.io import save

    save(plot_figure, '../outputs/decompose-analysis/latent-codes/latent-mapping-{}.html'.format(method))

/tmp/ipykernel_17279/1445940004.py:55: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(plot_figure, '../outputs/decompose-analysis/latent-codes/latent-mapping-{}.html'.format(method))
/tmp/ipykernel_17279/1445940004.py:55: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  save(plot_figure, '../outputs/decompose-analysis/latent-codes/latent-mapping-{}.html'.format(method))
/tmp/ipykernel_17279/1445940004.py:55: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(plot_figure, '../outputs/decompose-analysis/latent-codes/latent-mapping-{}.html'.format(method))
/tmp/ipykernel_17279/1445940004.py:55: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  save(plot_figure, '../outputs/